# Georreferenciamento e análise de informações do banco de sentenças do TJSP

### José Eduardo de Souza Pimentel - 2026

In [1]:
# Importações necessárias

import os
import json
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# Carrega variáveis de ambiente

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [1]:
# Configurações

MODELO = "gpt-5-nano" #"gpt-5-mini"
PATH_SENTENCAS = "dados"
MAX_INTERACOES = 10
CSV_SENTENCAS = 'dados\\sentencas.csv'
CSV_SENTENCAS_ANALISADAS = 'dados\\sentencas_analisadas.csv'
CSV_SENTENCAS_COM_ENDERECOS = 'dados\\sentencas_com_enderecos.csv'
CSV_SENTENCAS_GEORREFERENCIADAS = 'dados\\sentencas_georreferenciadas.csv'

# Limites geográficos (cantos do retângulo)
PONTO_1 = (-22.59505, -47.87594)
PONTO_2 = (-22.86262, -47.41858)

In [26]:
# Parametrização da requisição ao TJSP

PARAMS_TJSP = params = {
    "conversationId": "",
    "dadosConsulta.pesquisaLivre": "",
    "tipoNumero": "UNIFICADO",
    "numeroDigitoAnoUnificado": "",
    "foroNumeroUnificado": "",
    "dadosConsulta.nuProcesso": "",
    "dadosConsulta.nuProcessoAntigo": "",
    "classeTreeSelection.values": "",
    "classeTreeSelection.text": "",
    "assuntoTreeSelection.values": "3608",
    "assuntoTreeSelection.text": "Tráfico de Drogas e Condutas Afins",
    "agenteSelectedEntitiesList": "",
    "contadoragente": "0",
    "contadorMaioragente": "0",
    "cdAgente": "",
    "nmAgente": "",
    "dadosConsulta.dtInicio": "",
    "dadosConsulta.dtFim": "",
    "varasTreeSelection.values": "451-10,451-8,451-7101,451-7084,451-7094,451-7095,451-9,451-5623",
    "varasTreeSelection.text": "8 Registros selecionados",
    "dadosConsulta.ordenacao": "DESC"
}

# Download das sentenças

In [27]:
# Cria uma sessão persistente
session = requests.Session()

# 1a requisição para obter os cookies
response = session.get("https://esaj.tjsp.jus.br/cjpg/pesquisar.do", params=PARAMS_TJSP)

if response.status_code == 200:
    for i in range(1, MAX_INTERACOES + 1):
        pagina = str(i)
        url_pagina = f"https://esaj.tjsp.jus.br/cjpg/trocarDePagina.do?pagina={pagina}&conversationId="

        # Usa a mesma sessão para fazer a requisição
        response = session.get(url_pagina)

        if response.status_code == 200:
            # Salva o conteúdo da página
            with open(f"{PATH_SENTENCAS}/pagina_{pagina}.html", "wb") as file:
                file.write(response.content)
            print(f"Página {pagina} salva com sucesso.")
        else:
            print(f"Erro ao acessar a página {pagina}: {response.status_code}")

        # Intervalo entre requisições
        time.sleep(1)
else:
    print("Erro ao acessar a página inicial.")

Página 1 salva com sucesso.
Página 2 salva com sucesso.
Página 3 salva com sucesso.


# Criação do dataframe

In [28]:
def extrair_informacoes_html(html_path):
    processos = []
    with open(html_path, "r", encoding="utf-8") as file:
        soup = BeautifulSoup(file, "html.parser")
        textos = soup.find_all(class_="fundocinza1")
        for texto in textos:
            processo = {}
            # Número do Processo
            processo['numero_do_processo'] = texto.find('span', class_='fonteNegrito').text.strip()

            # Classe, Assunto, Magistrado, Comarca, Foro, Vara, Data de Disponibilização
            for row in texto.find_all('tr', class_='fonte'):
                if row.find('strong'):
                    key = row.find('strong').text.strip().replace(':', '').lower().replace(' ', '_')
                    value = row.find('strong').next_sibling.strip()
                    processo[key] = value

            # Sentença
            sentenca_divs = texto.find_all('div', align='justify')
            sentenca = ''
            for div in sentenca_divs:
                span = div.find('span')
                if span:
                    sentenca += span.text.strip() + '\n'
            processo['sentenca'] = sentenca.strip()

            processos.append(processo)

        return processos

In [29]:
data = []

for filename in os.listdir(PATH_SENTENCAS):
    if filename.endswith(".html"):
        filepath = os.path.join(PATH_SENTENCAS, filename)
        data += (extrair_informacoes_html(filepath))

# Cria o DataFrame
df = pd.DataFrame(data)
df.head(5)

,numero_do_processo,classe,assunto,comarca,foro,vara,data_de_disponibilização,sentenca,magistrado
0,1501036-78.2025.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,09/01/2026,,NaN
1,1501473-22.2025.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,2ª Vara Criminal,09/01/2026,,NaN
2,1501416-04.2025.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,09/01/2026,,NaN
3,1501498-35.2025.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,1ª Vara Criminal,09/01/2026,,NaN
4,1501733-02.2025.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,1ª Vara Criminal,08/01/2026,,NaN
5,1501391-93.2022.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,2ª Vara Criminal,07/01/2026,,NaN
6,1501551-84.2023.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,3ª Vara Criminal,07/01/2026,SENTENÇA\n\nProcesso Digital nº:\t1501551-84.2...,Rodrigo Pares Andreucci
7,1500825-76.2024.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,05/01/2026,SENTENÇA\n\nProcesso Digital nº:\t1500825-76.2...,GABRIELA DE ALMEIDA VERGUEIRO
8,1501156-92.2023.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,29/12/2025,SENTENÇA\n\nProcesso Digital nº:\t1501156-92.2...,GABRIELA DE ALMEIDA VERGUEIRO
9,1501774-03.2024.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,3ª Vara Criminal,19/12/2025,SENTENÇA\n\nProcesso Digital nº:\t1501774-03.2...,Rodrigo Pares Andreucci


In [8]:
# Cria o dataframe com sentenças não vazias e salva em CSV
df_1 = df[(df['sentenca']!='')]
df_1 = df_1.drop_duplicates(subset=['numero_do_processo'], keep='last') # Despreza os processos repetidos
df_1.reset_index(drop=True, inplace=True)
df_1.to_csv(CSV_SENTENCAS, index=False, encoding='utf-8-sig')  # utf-8-sig para compatibilidade com Excel
print(f"Arquivo CSV salvo em: {CSV_SENTENCAS}")

Arquivo CSV salvo em: dados\sentencas.csv


# Consulta à API OpenAI para análise das sentenças

In [36]:
SYSTEM_MSG = (
    "Você é assessor jurídico especializado em direito penal. "
    "Responda exclusivamente em JSON válido, sem comentários, com as chaves: "
    "'data_fato', 'quantidade_reus', 'endereco_fato', 'responsavel_prisao', "
    "'modus_operandi_reu', 'alegacao_reu', 'resultado_processo', 'resumo_sentenca'."
)

USER_TEMPLATE = (
    "Analise a sentença a seguir e devolva um objeto JSON contendo as chaves já indicadas."
    "Para a chave 'data_fato' responda apenas com a data do fato (ex.: 24/03/2025)"
    "ou escreva 'Não se aplica'."
    "Para a chave 'quantidade_reus' responda apenas com um número (ex.: 1, 2, 3 etc)"
    "ou escreva 'Não se aplica'."
    "Para a chave 'endereco_fato' responda com o endereço principal do fato "
    "(ex.: rua Alvorada, 123, bairro Jardim Brasília, Piracicaba/SP)."
    "ou escreva 'Não se aplica'."
    "Para a chave 'responsavel_prisao', informe que tipo de agente público realizou a prisão "
    "(ex.: policial militar, policial civil, policial penal, guarda municipal, outros) "
    "ou escreva 'Não se aplica'."
    "Para a chave 'modus_operandi_reu', resuma em até 70 palavras de que forma o tráfico era realizado "
    "ou escreva 'Não se aplica'."
    "Para a chave 'alegacao_reu', resuma em até 70 palavras o que o réu ou os réus alegaram em suas defesas "
    "ou escreva 'Não se aplica'."
    "Para a chave 'resultado_processo', responda 'Procedente', 'Improcedente', 'Parcialmente procedente' "
    "ou escreva 'Não se aplica'."
    "Para a chave 'resumo_sentenca', resuma em até 100 palavras a sentença em sua totalidade."
    "RESTRIÇÕES: para todos os casos, não se refira ao(s) réu(s) pelo nome. Use suas iniciais.\n\n"
    "SENTENÇA:\n\n{sentenca}"
)


client = OpenAI()

def analisar_sentenca_openai(sentenca: str) -> dict:
    """
    Analisa o texto de uma sentença usando modelo da OpenAI
    e retorna os dados extraídos em formato de dicionário Python.
    """

    resposta_api = None

    try:
        response = client.chat.completions.create(
            model=MODELO,
            response_format={"type": "json_object"},  # força o JSON
            messages=[
                {"role": "system", "content": SYSTEM_MSG},
                {"role": "user", "content": USER_TEMPLATE.format(sentenca=sentenca)}
            ]
        )

        if (
            response
            and response.choices
            and response.choices[0].message
            and response.choices[0].message.content
        ):
            resposta_api = response.choices[0].message.content.strip()
        else:
            resposta_api = "Erro: Resposta da API em formato inesperado."
            print(f"Resposta inesperada da API: {response}")

    except Exception as e:
        resposta_api = f"Erro ao processar com OpenAI: {e}"

    return resposta_api

In [37]:
lista_de_respostas = []
for idx, row in df_1.iterrows():
    resposta_texto = analisar_sentenca_openai(row['sentenca'])  # string retornada pela API
    try:
        analise = json.loads(resposta_texto)  # converte a string JSON para dict
    except Exception as e:
        print(f"Erro ao converter JSON no processo {row['numero_do_processo']}: {e}")
        print("Resposta bruta da API:", resposta_texto)
        # fallback: cria dicionário com valores padrão para não quebrar o pipeline
        analise = {
            "data_fato": "Não se aplica",
            "quantidade_reus": "Não se aplica",
            "endereco_fato": "Não se aplica",
            "responsavel_prisao": "Não se aplica",
            "modus_operandi_reu": "Não se aplica",
            "alegacao_reu": "Não se aplica",
            "resultado_processo": "Não se aplica",
            "resumo_sentenca": "Não se aplica"
        }
    n_processo = row['numero_do_processo']
    print(f"Analisando o processo {n_processo}...")
    lista_de_respostas.append((n_processo, analise))

Analisando o processo 1501551-84.2023.8.26.0599...
Analisando o processo 1500825-76.2024.8.26.0599...
Analisando o processo 1501156-92.2023.8.26.0599...
Analisando o processo 1501774-03.2024.8.26.0599...
Analisando o processo 1500518-25.2024.8.26.0599...
Analisando o processo 1501896-79.2025.8.26.0599...
Analisando o processo 1501366-75.2025.8.26.0599...
Analisando o processo 1501824-97.2022.8.26.0599...
Analisando o processo 1501966-67.2023.8.26.0599...
Analisando o processo 1501612-42.2023.8.26.0599...
Analisando o processo 0005542-09.2020.8.26.0451...
Analisando o processo 1501992-65.2023.8.26.0599...
Analisando o processo 1501305-88.2023.8.26.0599...
Analisando o processo 1501564-83.2023.8.26.0599...
Analisando o processo 1502021-18.2023.8.26.0599...
Analisando o processo 1500799-15.2023.8.26.0599...
Analisando o processo 1500335-88.2023.8.26.0599...
Analisando o processo 1500533-96.2021.8.26.0599...
Analisando o processo 1501935-81.2022.8.26.0599...
Analisando o processo 1501289-7

In [38]:
# Cria um DataFrame a partir da lista de respostas
# A lista_de_respostas contém tuplas (numero_do_processo, dict_analise)
# Vamos expandir o dicionário em colunas do DataFrame

data_para_df = []
for numero_processo, analise_dict in lista_de_respostas:
  row_data = {'numero_do_processo': numero_processo}
  row_data.update(analise_dict) # Adiciona as chaves do dicionário como colunas
  data_para_df.append(row_data)

In [39]:
df_respostas = pd.DataFrame(data_para_df)
display(df_respostas.head(5))

df_respostas.to_csv(CSV_SENTENCAS_ANALISADAS, index=False, encoding='utf-8-sig')
print(f"Arquivo CSV salvo em: {CSV_SENTENCAS_ANALISADAS}")

,numero_do_processo,data_fato,quantidade_reus,endereco_fato,responsavel_prisao,modus_operandi_reu,alegacao_reu,resultado_processo,resumo_sentenca
0,1501551-84.2023.8.26.0599,Não se aplica,1,Não se aplica,Não se aplica,Tráfico realizado com venda de entorpecentes e...,Não se aplica,Procedente,Resumo da sentença: Ação de tráfico de drogas ...
1,1500825-76.2024.8.26.0599,03/05/2024,1,"Rua Querubim Sampaio, próximo ao nº 1, bairro ...",policial militar,"R.C., em conluio com o menor A.B.B., aproximou...",A defesa sustenta absolvição por insuficiência...,Procedente,Trata-se de ação penal por tráfico de drogas c...
2,1501156-92.2023.8.26.0599,31/05/2023,1,"Rua do Enxofre, 963, bairro Enxofre, Piracicab...",guarda civil municipal,"Em via pública, na região conhecida pelo tráfi...",A defesa pleiteou a absolvição ou a desclassif...,Procedente,O.J.S. foi condenado pela prática de tráfico d...
3,1501774-03.2024.8.26.0599,25/10/2024,3,"Rua Lins, 288, Itapuã, Piracicaba/SP",policial militar,"Em via pública, com unidade de desígnios, os r...",A defesa alegou ausência de prova suficiente d...,Procedente,Julgo procedente a ação e condeno JAFDS e CALD...
4,1500518-25.2024.8.26.0599,13/03/2024,1,"Rua Campos Sales, 680, apto 212, Bairro São Ju...",policial civil,"Guardava entorpecentes para tráfico, com porçõ...",Preliminar: nulidade da prova obtida com acess...,Procedente,"Julgando procedente a denúncia, o juízo conden..."


Arquivo CSV salvo em: dados\sentencas_analisadas.csv


In [40]:
df_geral = pd.merge(df_1, df_respostas, on='numero_do_processo')
df_geral.head(5)

,numero_do_processo,classe,assunto,comarca,foro,vara,data_de_disponibilização,sentenca,magistrado,data_fato,quantidade_reus,endereco_fato,responsavel_prisao,modus_operandi_reu,alegacao_reu,resultado_processo,resumo_sentenca
0,1501551-84.2023.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,3ª Vara Criminal,07/01/2026,SENTENÇA\n\nProcesso Digital nº:\t1501551-84.2...,Rodrigo Pares Andreucci,Não se aplica,1,Não se aplica,Não se aplica,Tráfico realizado com venda de entorpecentes e...,Não se aplica,Procedente,Resumo da sentença: Ação de tráfico de drogas ...
1,1500825-76.2024.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,05/01/2026,SENTENÇA\n\nProcesso Digital nº:\t1500825-76.2...,GABRIELA DE ALMEIDA VERGUEIRO,03/05/2024,1,"Rua Querubim Sampaio, próximo ao nº 1, bairro ...",policial militar,"R.C., em conluio com o menor A.B.B., aproximou...",A defesa sustenta absolvição por insuficiência...,Procedente,Trata-se de ação penal por tráfico de drogas c...
2,1501156-92.2023.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,29/12/2025,SENTENÇA\n\nProcesso Digital nº:\t1501156-92.2...,GABRIELA DE ALMEIDA VERGUEIRO,31/05/2023,1,"Rua do Enxofre, 963, bairro Enxofre, Piracicab...",guarda civil municipal,"Em via pública, na região conhecida pelo tráfi...",A defesa pleiteou a absolvição ou a desclassif...,Procedente,O.J.S. foi condenado pela prática de tráfico d...
3,1501774-03.2024.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,3ª Vara Criminal,19/12/2025,SENTENÇA\n\nProcesso Digital nº:\t1501774-03.2...,Rodrigo Pares Andreucci,25/10/2024,3,"Rua Lins, 288, Itapuã, Piracicaba/SP",policial militar,"Em via pública, com unidade de desígnios, os r...",A defesa alegou ausência de prova suficiente d...,Procedente,Julgo procedente a ação e condeno JAFDS e CALD...
4,1500518-25.2024.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,1ª Vara Criminal,17/12/2025,SENTENÇA\n\nProcesso Digital nº:\t1500518-25.2...,Ana Claudia Madeira de Oliveira,13/03/2024,1,"Rua Campos Sales, 680, apto 212, Bairro São Ju...",policial civil,"Guardava entorpecentes para tráfico, com porçõ...",Preliminar: nulidade da prova obtida com acess...,Procedente,"Julgando procedente a denúncia, o juízo conden..."


In [41]:
df_geral_com_endereco = df_geral[df_geral['endereco_fato'] != 'Não se aplica']
df_geral_com_endereco.head(5)

df_geral_com_endereco.to_csv(CSV_SENTENCAS_COM_ENDERECOS, index=False, encoding='utf-8-sig')
print(f"Arquivo CSV salvo em: {CSV_SENTENCAS_COM_ENDERECOS}")

Arquivo CSV salvo em: dados\sentencas_com_enderecos.csv


# Georreferenciamento do dataframe

In [42]:
def buscar_coords(endereco, api_key):
    """Função auxiliar para retornar latitude e longitude ou None em caso de erro."""
    if not endereco:
        return None, None

    url = "https://maps.googleapis.com/maps/api/geocode/json"
    params = {"address": endereco, "key": api_key, "language": "pt-BR"}

    try:
        response = requests.get(url, params=params)
        data = response.json()

        if data.get("status") == "OK":
            location = data["results"][0]["geometry"]["location"]
            return location["lat"], location["lng"]
        else:
            return None, None
    except:
        return None, None

In [43]:
df_geral_com_endereco[['latitude', 'longitude']] = df_geral_com_endereco['endereco_fato'].apply(
    lambda x: pd.Series(buscar_coords(x, GOOGLE_API_KEY))
)

C:\Users\jepim\AppData\Local\Temp\ipykernel_6780\74596199.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_geral_com_endereco[['latitude', 'longitude']] = df_geral_com_endereco['endereco_fato'].apply(
C:\Users\jepim\AppData\Local\Temp\ipykernel_6780\74596199.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_geral_com_endereco[['latitude', 'longitude']] = df_geral_com_endereco['endereco_fato'].apply(


In [44]:
df_geral_com_endereco.head(5)

,numero_do_processo,classe,assunto,comarca,foro,vara,data_de_disponibilização,sentenca,magistrado,data_fato,quantidade_reus,endereco_fato,responsavel_prisao,modus_operandi_reu,alegacao_reu,resultado_processo,resumo_sentenca,latitude,longitude
1,1500825-76.2024.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,05/01/2026,SENTENÇA\n\nProcesso Digital nº:\t1500825-76.2...,GABRIELA DE ALMEIDA VERGUEIRO,03/05/2024,1,"Rua Querubim Sampaio, próximo ao nº 1, bairro ...",policial militar,"R.C., em conluio com o menor A.B.B., aproximou...",A defesa sustenta absolvição por insuficiência...,Procedente,Trata-se de ação penal por tráfico de drogas c...,-22.745242,-47.675811
2,1501156-92.2023.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,29/12/2025,SENTENÇA\n\nProcesso Digital nº:\t1501156-92.2...,GABRIELA DE ALMEIDA VERGUEIRO,31/05/2023,1,"Rua do Enxofre, 963, bairro Enxofre, Piracicab...",guarda civil municipal,"Em via pública, na região conhecida pelo tráfi...",A defesa pleiteou a absolvição ou a desclassif...,Procedente,O.J.S. foi condenado pela prática de tráfico d...,-22.744447,-47.656967
3,1501774-03.2024.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,3ª Vara Criminal,19/12/2025,SENTENÇA\n\nProcesso Digital nº:\t1501774-03.2...,Rodrigo Pares Andreucci,25/10/2024,3,"Rua Lins, 288, Itapuã, Piracicaba/SP",policial militar,"Em via pública, com unidade de desígnios, os r...",A defesa alegou ausência de prova suficiente d...,Procedente,Julgo procedente a ação e condeno JAFDS e CALD...,-22.753135,-47.661101
4,1500518-25.2024.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,1ª Vara Criminal,17/12/2025,SENTENÇA\n\nProcesso Digital nº:\t1500518-25.2...,Ana Claudia Madeira de Oliveira,13/03/2024,1,"Rua Campos Sales, 680, apto 212, Bairro São Ju...",policial civil,"Guardava entorpecentes para tráfico, com porçõ...",Preliminar: nulidade da prova obtida com acess...,Procedente,"Julgando procedente a denúncia, o juízo conden...",-22.717498,-47.645577
5,1501896-79.2025.8.26.0599,Procedimento Especial da Lei Antitóxicos,Tráfico de Drogas e Condutas Afins,Piracicaba,Foro de Piracicaba,4ª Vara Criminal,17/12/2025,SENTENÇA\n\nProcesso Digital nº:\t1501896-79.2...,Gisela Ruffo,09/08/2025,3,"Rua Jumirim, 161, Matão, Piracicaba/SP",policial militar,"RPO, DDC e LFP atuavam conjuntamente no endere...",DDC confessou a prática. RPO e LFP negaram env...,Procedente,Condenação de três réus por tráfico de drogas ...,-22.748460,-47.671598


In [45]:
# Salvar o DataFrame georreferenciado em um novo arquivo CSV
df_geral_com_endereco.to_csv(CSV_SENTENCAS_GEORREFERENCIADAS, index=False, encoding='utf-8-sig')
print(f"DataFrame de respostas salvo em: {CSV_SENTENCAS_GEORREFERENCIADAS}")

DataFrame de respostas salvo em: dados\sentencas_georreferenciadas.csv


In [ ]:
# Cálculo dos limites min/max para latitude e longitude
min_lat = min(PONTO_1[0], PONTO_2[0])
max_lat = max(PONTO_1[0], PONTO_2[0])
min_lon = min(PONTO_1[1], PONTO_2[1])
max_lon = max(PONTO_1[1], PONTO_2[1])

# Filtragem dos dados dentro do retângulo
df_geral_com_endereco_filtrado = df_geral_com_endereco[
    (df_geral_com_endereco['latitude'] >= min_lat) & (df_geral_com_endereco['latitude'] <= max_lat) &
    (df_geral_com_endereco['longitude'] >= min_lon) & (df_geral_com_endereco['longitude'] <= max_lon)
    ]

In [46]:
# Gera o arquivo para a visualização
df_geral_com_endereco_filtrado[['numero_do_processo', 'data_fato', 'responsavel_prisao', 'modus_operandi_reu', 
                       'alegacao_reu', 'resultado_processo', 'resumo_sentenca', 'latitude', 'longitude']].to_csv(
    'dados.csv', index=False, encoding='utf-8-sig')